In [9]:
import tweepy
from kafka import KafkaConsumer
from kafka import KafkaProducer
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import Stream
import json

# Set consumer key, consumer secret, access token and access token secret
# Read them from your Twitter dev page
consumer_key = "24Jgnez2AMO39yWzvZT5rdA8m"
consumer_secret = "dcVJLFGAgg2DL9NqGZbEXdEb2nMuXJItos3jdbaaj4PMOMF1oI"
access_token = "1334438191668072448-omagasI8laVHrBjVA6GTQa4hwFuvk7"
access_token_secret = "FZjCnP074OCPVOs7qulXFVMN7bfUThEgPz1m0fqbg3vlb"

# Authenticate with the OAuth protocol
# See: https://en.wikipedia.org/wiki/OAuth
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Get the API object passing the authentication object
# See http://docs.tweepy.org/en/latest/api.html to discover the available methods
api = tweepy.API(auth)

producer = KafkaProducer(
  bootstrap_servers=["kafka:9092"],
  value_serializer=lambda v: json.dumps(v).encode("utf-8"))

consumer = KafkaConsumer(
  "sinotweet",
  bootstrap_servers=["kafka:9092"],
  auto_offset_reset="earliest",
  client_id="jupyterlab-01",
  group_id="jupyterlab",
  metadata_max_age_ms=10000,
  value_deserializer=lambda v: json.loads(v.decode("utf-8")))

# We "continuously stream" from the consumer object
for tweet in consumer:
  enriched_tweet = tweet.value
  enriched_tweet['followers_ids'] = [1,2,3]
  producer.send(topic='enriched-tweets-sino', value=enriched_tweet)

KeyboardInterrupt: 